In this analysis, we're going to filter the global dataset of tech layoffs to focus on the United States market, from **2020** to **2025**.

As a first step, we need the **Pandas** library for this analysis.

In [59]:
import pandas as pd

This prevents pandas from showing scientific notation (example : 1.5e6) which is hard to read.

In [60]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

Now let's load the raw dataset from the csv file into the pandas DataFrame.

In [61]:
df = pd.read_csv('layoffs.csv')

We need to verify if the data loaded correctly, so let's check the first 5 rows.

In [62]:
df.head()

,company,location,total_laid_off,date,percentage_laid_off,industry,source,stage,funds_raised,country,date_added
0,Modern Hydrogen,Seattle,NaN,12/3/2025,NaN,Energy,https://fuelcellsworks.com/2025/12/03/green-in...,Series B,99.00,United States,12/4/2025
1,Aqua Security,"Tel Aviv, Non-U.S.",20.00,12/1/2025,NaN,Security,https://www.calcalistech.com/ctechnews/article...,Series E,325.00,United States,12/2/2025
2,Inbound Health,Minneapolis,NaN,12/1/2025,1.00,Healthcare,https://www.axios.com/pro/health-tech-deals/20...,Series B,50.00,United States,12/2/2025
3,Shopify,"Ottawa, Non-U.S.",NaN,11/26/2025,NaN,Retail,https://www.thecanadianpressnews.ca/business/s...,Post-IPO,122.00,Canada,12/4/2025
4,HP,SF Bay Area,4000.00,11/25/2025,NaN,Hardware,https://www.wsj.com/tech/hp-to-cut-up-to-10-of...,Post-IPO,NaN,United States,3/1/2025


We need to check for missing values in critical columns.

In [63]:
df.isnull().sum()

,0
company,0
location,1
total_laid_off,1465
date,0
percentage_laid_off,1552
industry,2
source,3
stage,5
funds_raised,481
country,2


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4222 entries, 0 to 4221
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              4222 non-null   object 
 1   location             4221 non-null   object 
 2   total_laid_off       2757 non-null   float64
 3   date                 4222 non-null   object 
 4   percentage_laid_off  2670 non-null   float64
 5   industry             4220 non-null   object 
 6   source               4219 non-null   object 
 7   stage                4217 non-null   object 
 8   funds_raised         3741 non-null   float64
 9   country              4220 non-null   object 
 10  date_added           4222 non-null   object 
dtypes: float64(3), object(8)
memory usage: 363.0+ KB


We need to convert the **'Date'** column from **object (string)** to **datetime** format, this allows us to perform time series analysis.

In [65]:
df['date'] = pd.to_datetime(df['date'])

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4222 entries, 0 to 4221
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   company              4222 non-null   object        
 1   location             4221 non-null   object        
 2   total_laid_off       2757 non-null   float64       
 3   date                 4222 non-null   datetime64[ns]
 4   percentage_laid_off  2670 non-null   float64       
 5   industry             4220 non-null   object        
 6   source               4219 non-null   object        
 7   stage                4217 non-null   object        
 8   funds_raised         3741 non-null   float64       
 9   country              4220 non-null   object        
 10  date_added           4222 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 363.0+ KB


We need to drop rows where **'total_laid_off'** is missing : **"NaN"**, we cannot analyze workforce reduction trends if the core metric is missing.

In [67]:
df_clean = df.dropna(subset=['total_laid_off'])

In [68]:
df_clean.isnull().sum()

,0
company,0
location,0
total_laid_off,0
date,0
percentage_laid_off,862
industry,1
source,0
stage,1
funds_raised,278
country,1


Now we need to calculate the grand total of global layoffs in the entire dataset.

In [69]:
total_layoffs = df_clean['total_laid_off'].sum()

In [70]:
print(f"Total Global Layoffs: {total_layoffs:,.0f}")

Total Global Layoffs: 799,676


Let's identify the top 5 companies with the highest total layoffs, we're going to group by company name, sum the layoffs, sort descending and taking top 5.

In [71]:
top_companies = df_clean.groupby('company')['total_laid_off'].sum().sort_values(ascending=False).head(5)

In [72]:
print(top_companies)

company
Intel        43115.00
Amazon       41940.00
Microsoft    30055.00
Meta         25300.00
Salesforce   16525.00
Name: total_laid_off, dtype: float64


Now let's analyze which industries were hit the hardest, grouping by industry, sum layoffs, sort descending, and taking top 10.

In [73]:
industry_trends = df_clean.groupby('industry')['total_laid_off'].sum().sort_values(ascending=False)

In [74]:
print(industry_trends.head(10))

industry
Hardware         93127.00
Retail           88061.00
Other            85015.00
Consumer         80436.00
Transportation   64866.00
Finance          52889.00
Food             50174.00
Healthcare       38844.00
Travel           23370.00
Sales            21063.00
Name: total_laid_off, dtype: float64


  Now let's filter the **United States**, one of the countries included in this dataset, for a focused analysis.

In [75]:
us_check = df_clean[df_clean['country'].isin(['United States'])]

For the final goal of our analysis, we're going to filter the **total of tech layoffs** by company in the **United States**, from **2020** to **2025**.

In [76]:
print("\n--- United States Data ---")
print(us_check[['company', 'country', 'total_laid_off', 'date']])


--- United States Data ---
              company        country  total_laid_off       date
1       Aqua Security  United States           20.00 2025-12-01
4                  HP  United States         4000.00 2025-11-25
5             ezCater  United States           40.00 2025-11-21
6     Monarch Tractor  United States          102.00 2025-11-19
12               Pipe  United States          200.00 2025-11-16
...               ...            ...             ...        ...
4216         Help.com  United States           16.00 2020-03-16
4218     HopSkipDrive  United States            8.00 2020-03-13
4219      Panda Squad  United States            6.00 2020-03-13
4220    Tamara Mellon  United States           20.00 2020-03-12
4221         EasyPost  United States           75.00 2020-03-11

[1703 rows x 4 columns]


As a conclusion, this analysis **filtered the global dataset to focus on the United States market.**
Key Findings:
1. **Volume:** The US accounts for 1,703 individual layoff events (a significant portion of the global total).
2. **Timeline:** The data captures a 5-year trend (2020-2025), highlighting that this is a long-term structural shift, not just a short-term spike.
3. **Severity:** High-volume layoffs in major entities (like HP) indicate enterprise-level restructuring.